In [1]:
import pandas as pd
import numpy as np
from ANNbuilder import build_network
from pso import pso_optimization
from loss import mse_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [52]:
data = pd.read_csv(
    "/Users/ilya/Desktop/GitHub_Repositories/HW_University/Data_Mining/datasets/Real estate.csv")
data.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [53]:
data = data.drop(['No'], axis=1)

In [54]:
data.shape

(414, 7)

In [55]:
X = data.drop("Y house price of unit area", axis=1).values
y = data["Y house price of unit area"].values.reshape(-1, 1)

In [56]:
X[:5]

array([[2012.917  ,   32.     ,   84.87882,   10.     ,   24.98298,
         121.54024],
       [2012.917  ,   19.5    ,  306.5947 ,    9.     ,   24.98034,
         121.53951],
       [2013.583  ,   13.3    ,  561.9845 ,    5.     ,   24.98746,
         121.54391],
       [2013.5    ,   13.3    ,  561.9845 ,    5.     ,   24.98746,
         121.54391],
       [2012.833  ,    5.     ,  390.5684 ,    5.     ,   24.97937,
         121.54245]])

In [57]:
y[:5]

array([[37.9],
       [42.2],
       [47.3],
       [54.8],
       [43.1]])

In [58]:
# Data separation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Data scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [62]:
# Defining the network architecture and activation functions
from sklearn.metrics import mean_squared_error
architecture = [6, 12, 1]  # Network architecture
activations = ['relu', 'linear']  # Activation functions

# Use the build_network function to create the network
network = build_network(architecture, activations)

# Get the dimensionality of the network weights vector
dimensions = network.total_weights()

# Call pso_optimization function with the created network and parameters
best_weights = pso_optimization(
    num_particles=100,
    num_iterations=300,
    loss_func=mse_loss,
    network=network,
    train_data=X_train_scaled,
    train_labels=y_train,
    dimensions=dimensions)

# Set the best weights to the network
network.set_weights(best_weights)

# Test the network
output = network.forward(X_test_scaled)
predictions = output

# Print the results
print("Test RMSE:", mean_squared_error(y_test, predictions)**0.5)

Iteration 25/300, Current Best Loss: 213.15221605233404
Iteration 50/300, Current Best Loss: 146.36627825620246
Iteration 75/300, Current Best Loss: 115.46759425392254
Iteration 100/300, Current Best Loss: 92.51574165103584
Iteration 125/300, Current Best Loss: 75.66923843685264
Iteration 150/300, Current Best Loss: 69.85866439552213
Iteration 175/300, Current Best Loss: 66.77907973557684
Iteration 200/300, Current Best Loss: 64.27196788137742
Iteration 225/300, Current Best Loss: 62.537015954539854
Iteration 250/300, Current Best Loss: 61.39392799438032
Iteration 275/300, Current Best Loss: 60.20263092570067
Iteration 300/300, Current Best Loss: 59.290856832382275
Final Best Loss: 59.290856832382275 on Iteration: 298
Test RMSE: 6.302491120523832


In [61]:
data["Y house price of unit area"].describe()

count    414.000000
mean      37.980193
std       13.606488
min        7.600000
25%       27.700000
50%       38.450000
75%       46.600000
max      117.500000
Name: Y house price of unit area, dtype: float64